In [10]:
import os
from sys import getsizeof
import tensorflow as tf
import base64
import numpy as np
from googleapiclient import discovery
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
TFR_TEST = 'test.tfrecord'
BUCKET = 'gs://robolab/'

NUM_CLASSES = 2

IMG_HEIGHT = 80
IMG_WIDTH = 71

In [3]:
def predict_json(project, model, key, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        key: private key with credentials
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    
    scopes = ['https://www.googleapis.com/auth/cloud-platform']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(key, scopes=scopes)
    service = discovery.build('ml', 'v1', credentials=credentials)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [71]:
def get_image_id(serialized_example):

    features = tf.parse_single_example(
        serialized_example,
        features={'image_raw': tf.FixedLenFeature([], tf.string),
                  'image_id': tf.FixedLenFeature([], tf.string)})
    
    return features['image_id']

In [72]:
# on free GCP trials HTTP payload size is limited to 1.5 MB
# batch size would depend on number of tf.Examples used
# so we do batch of 10 examples only
instances = []
image_id_tensors = []
preds = []

record_iterator = tf.python_io.tf_record_iterator(path=BUCKET + TFR_TEST)

i = 0
for string_record in record_iterator:
    
    image_id_tensors.append(get_image_id(string_record))
    
    if i > 10:
        break
    i += 1
    
    data = base64.b64encode(string_record).decode('utf-8')
    instances.append({"b64": data})

In [73]:
preds = predict_json(
    'psyched-timer-223917',
    'subvoc',
    'private_key.json',
    instances,
    version=None)

In [76]:
with tf.Session() as sess:
     image_ids = sess.run(image_id_tensors)

In [87]:
for image_id, pred in zip(image_ids, preds):
    image_id = image_id.decode('utf-8')
    print('id: {}, label: {}, probs: [{:.3f}, {:.3f}]'.format(
        image_id, pred['classes'], pred['probabilities'][0], pred['probabilities'][1]))

id: ZM5JX5MM, label: 1, probs: [0.012, 0.988]
id: WSR9JCDZ, label: 0, probs: [0.983, 0.017]
id: 3HZI0I41, label: 0, probs: [0.956, 0.044]
id: BU9XBHAC, label: 0, probs: [0.563, 0.437]
id: 4GF4OKQY, label: 0, probs: [0.971, 0.029]
id: X3WKUXRT, label: 1, probs: [0.038, 0.962]
id: Y88KZ0TJ, label: 0, probs: [0.967, 0.033]
id: 4OSS97LG, label: 1, probs: [0.078, 0.922]
id: WE2RASCM, label: 0, probs: [0.902, 0.098]
id: QA3FJR5F, label: 0, probs: [0.990, 0.010]
id: QDL3OILK, label: 0, probs: [0.981, 0.019]
